# 25-26: 4369 -- PROGRAMMING FOR DATA ANALYTICS
## Big Project

<img src="../images/Programming.png" alt="progrsmming" width="400">

One of the main challenges of this project (I found) was deciding on a suitable data subject to research and then attempting to **source** data.  
While I looked into data for Retail Sales, Traffic Accidents, Weather, etc (and browsing Kaggle for potential subjects) ... after several failed attempts, I found that *I needed a pursue a subject I was genuinely curious about.*  

I decided to look into the subject of **Trends in the use of Programming Languages** and to leverage the huge data repository of **GitHub** as a data source.  
  
I wanted to potentially look at the following Programming Language Trends
* Analyze which languages are growing/declining over time
* Analyze language switching patterns (Devs moving from language x to language y)
* Has the recent explosion of AI resulted in a rise of AI-related repositories 
    * How have these AI-related repositories grown in recent years
    * How does this growth compare with traditional languages
* Compare developer activity by geography and timezone
    * How does India and China compare with EU/US in github activity 
* Visualize the shift from older to newer languages


GitHub Data Sources are available in :  
* GitHub API - Free tier allows decent access  
* GH Archive - Historical GitHub event data (billions of events!)  
* GitHub's public datasets on Google BigQuery  
* Pre-made datasets on Kaggle  

I initially looked into the GitHub API as a mechanism to extract the datasets I would require.

***
## The GitHub REST API with Python
### (... and its limitations 😖)

I experimented a while with Python scripts that collect GitHub Metadata and save to CSV files.  
https://blog.apify.com/python-github-api/  
https://melaniesoek0120.medium.com/how-to-use-github-api-to-extract-data-with-python-bdc61106a501  

I also investigated the use of the PyGithub Library.   
https://github.com/PyGithub/PyGithub  
https://pygithub.readthedocs.io/en/latest/introduction.html  
https://stackoverflow.com/questions/10625190/most-suitable-python-library-for-github-api-v3  
https://www.youtube.com/watch?v=QaURSdmP0o8  

But, this only highlighted how restrictive this API's "Rate Limit" can be.  
Look at script below (generated with help of Claude AI) that uses pyGithub module to attempt to look at the count of language specific  repositories created with a specific period  

[My Python Script for Extracting Github Data](./testing/github_api_test.py)  
Note: To use the PyGithub Module you will need to install PyGithub library with  
`pip install PyGithub`  

If you look at the results of this script (output to [this text file](./testing/Github_REST_API_Results.txt)), all language for all months return exactly same result ... "1,000 repos"  
**I am obviously hitting a limit of some sort here.**
This is GitHub Search API’s “1,000-result cap”
   
A simplified version (with a hard-coded sample) is shown below where even a reduced time interval of only 3 hours is used...   
It still returns a count close to the limit of 1000 repositories   

*Experimentation with values for language, start_date, and end_date highlight how restrictive this 1000 limit is.*

In [4]:
from github import Github, RateLimitExceededException, Auth
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Load environment variables from .env file
load_dotenv()

# Get the token from environment variables
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")


# Create authentication object
auth = Auth.Token(GITHUB_TOKEN)
# Initialize GitHub client with auth
g = Github(auth=auth)

# Verify authentication
try:
    user = g.get_user()
    print(f"Authenticated as: {user.login}")
except Exception as e:
    print(f"Authentication failed: {e}")

# Example usage 
language = 'Python'
start_date = '2024-03-01T00:00:00+00:00'
end_date = '2024-03-01T03:00:00+00:00'

# Build the API search query
query = f"language:{language} created:{start_date}..{end_date}"
# Search repositories with PyGithub
result = g.search_repositories(query=query)
# Get total count
total_count = result.totalCount
print(f"Count of {language} repos created between {start_date} and {end_date}: {total_count}")



Authenticated as: ngn73
Count of Python repos created between 2024-03-01T00:00:00+00:00 and 2024-03-01T03:00:00+00:00: 940


<font color="crimson">... **I needed to find an alternative (less restrictive) method to extracting Github Repository metadata.**</font>   

<font color="yellow">But, there are still some advantages to using the GitHub API where this limit is not applicable to extracting geo-based data (more on that later)</font>   

***
## Google BigQuery + GitHub Archive  
  
As a potential alternative to using the GitHub REST API (with Python), Google's Cloud-based Data Warehouse Service "BigQuery" provides a series of tools for extracting and analyzing large scale databases (sometimes billions of rows)   
BigQuery abstracts the complexity of underlying server/database infrastructure. You just load data and run standard SQL queries against it.  
https://console.cloud.google.com/bigquery/  
https://www.datacamp.com/tutorial/beginners-guide-to-bigquery  
https://www.udemy.com/course/introduction-to-google-cloud-bigquery/  <font color="crimson">*(requires subscription)</font>

More importantly, Google is also in collaboration with GitHub providing access to an incredible new Github Archive database via **Google BigQuery**  
The Google BigQuery Public Datasets program now offers a full snapshot of the content of more than 2.8 million open source GitHub repositories in BigQuery  
This provides a alternative mechanism to analyze the source code of almost 2 billion files with a simple (or complex) SQL query without the limitations experienced above.   
  

https://cloud.google.com/blog/topics/public-datasets/github-on-bigquery-analyze-all-the-open-source-code  
https://hoffa.medium.com/github-on-bigquery-analyze-all-the-code-b3576fd2b150  
https://hoffa.medium.com/400-000-github-repositories-1-billion-files-14-terabytes-of-code-spaces-or-tabs-7cfe0b5dd7fd  
https://github.com/fhoffa/analyzing_github?tab=readme-ov-file  

The use of SQL is ideal as I am quite comfortable with the SQL syntax  
https://docs.cloud.google.com/bigquery/docs/reference/rest?apix=true   
https://codelabs.developers.google.com/codelabs/bigquery-github#0  

There is a lot of menus within Google Cloud. Within Google Cloud, goto Navigation Menu -> BigQuery->Studio and select 'SQL Query'
<img src="../images/Goole_BigQuery_SQL.png" alt="Google BigQuery" width="600">  

**SQL queries within BigQuery can be used to structure, clean, and prepare the data so it meets the needs of my analysis   
Results can then be saved as CSV or JSon files for further processing/visualizing with Python.**  
  
The core tables to query against fall under 2 categories:  
1. "githubarchive" Time based activity Tables (examples below)
* **githubarchive.day.20241201** (Archive of events for day December 1, 2024)  
* **githubarchive.month.202411** (Archive of events for Month November 2024)  
* **githubarchive.year.2024** (Archive of events for Year 2024)  

To query the whole of 2024 you can also use a wildcard in the Table name (e.g. githubarchive.day.2024*)   

2. "github_repos" Snapshots of GitHub repository contents, metadata, commits, etc.
* **commits**
*  **files**
*  **contents**
*  **languages**
   
 



The details of each Gitub event (push, pull, create, frk, etc.) are stored within the "payload" field. The content of the "payload" field is different for each event type and may be updated by GitHub at any point, hence it is kept as a serialized JSON string value in BigQuery. JSON_EXTRACT functions can be used to apply filters to this Field  
  
https://www.gharchive.org/#:~:text=The%20content%20of%20the%20%22payload,access%20data%20in%20this%20field.  
https://github.com/igrigorik/gharchive.org/issues/148  
  
  
An example SQL script below filters 'pull' events for Python and Javascript languages using **JSON_EXTRACT_SCALAR( )**

```
SELECT 
    FORMAT_DATE('%Y-%m', PARSE_DATE('%y%m%d', _TABLE_SUFFIX)) as month,
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.base.repo.language') as language,
    COUNT(*) as pr_count,
    COUNT(DISTINCT repo.name) as active_repos
FROM `githubarchive.day.20*`
WHERE 
    type = 'PullRequestEvent'
    AND JSON_EXTRACT_SCALAR(payload, '$.pull_request.base.repo.language') IN ('Python', 'JavaScript')
    AND _TABLE_SUFFIX BETWEEN 
        FORMAT_DATE('%y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH))
        AND FORMAT_DATE('%y%m%d', CURRENT_DATE())
GROUP BY month, language
ORDER BY month DESC, pr_count DESC;
```


It could also be argued that the full data analysis could be performed completely within the BigQuery Studio as it offers a wide range of advanced tools for Analysis.  
But, in the interest of displaying the use of a range of tools covered in Module *25-26: 4369 -- Programming for Data Analytics* with an emphasis on Python, I will export datasets from BigQuery as CSV/JSon and later process with Python code.

***
## Using BigQuery and Python
Python can integrate with BigQuery Datasources with the **google-cloud-bigquery** package.  
https://docs.cloud.google.com/python/docs/reference/bigquery/latest  
This will require the Setup of authentication that will require additional configuration (application credentials, project_id, etc.).  
But, in the interests of migrating code other machines (and allowing code to be potentially tested by Andrew), I would definitely be better off using the BigQuery console directly for simpler, manual exports of SQL Data Resultsets (into CSV or JSon format)  
  
**So I will use the BigQuery console directly to generate CSV or JSon result Files that will act as the Data-source for Analysis Project.**
***